In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.applications import MobileNetV2  # 1. Import the MobileNetV2 model
from tensorflow.keras.layers import AveragePooling2D # Import the AveragePooling2D layer
from tensorflow.keras.layers import Dropout # Import the Dropout layer
from tensorflow.keras.layers import Flatten # Import the Flatten layer
from tensorflow.keras.layers import Dense # Import the Dense layer
from tensorflow.keras.layers import Input # Import the Input layer
from tensorflow.keras.models import Model # Import the Model layer
from tensorflow.keras.optimizers.legacy import Adam  # 2. Import the Adam optimizer
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input  # 3. Import the preprocess_input function from the mobilenet_v2 module
from tensorflow.keras.preprocessing.image import img_to_array  # 4. Import the img_to_array function from the keras.preprocessing.image module
from tensorflow.keras.preprocessing.image import load_img  # 5. Import the load_img function from the keras.preprocessing.image module
from tensorflow.keras.utils import to_categorical  # 6. Import the to_categorical function from the keras.utils module
from sklearn.preprocessing import LabelBinarizer  # 7. Import the LabelBinarizer class from the sklearn.preprocessing module
from sklearn.model_selection import train_test_split  # 8. Import the train_test_split function from the sklearn.model_selection module
from sklearn.metrics import classification_report  # 9. Import the classification_report function from the sklearn.metrics module
from imutils import paths  # 10. Import the paths module from the imutils package
import matplotlib.pyplot as plt  # 11. Import the pyplot module from the matplotlib package
import numpy as np  # 12. Import the numpy module 
import argparse  # 13. Import the argparse module
import os  # 14. Import the os module
import scipy  # 15. Import the scipy module
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

In [2]:
import argparse

args = {
    "dataset": "Dataset",
    "plot": "plot.png",
    "model": "mask_detector.model"
}

In [3]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-4  # 1. Set the initial learning rate to 0.0001
EPOCHS = 20  # 2. Set the number of epochs to 20
BS = 32  # 3. Set the batch size to 32

In [4]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")  # 4. Print a message to indicate that images are being loaded
imagePaths = list(paths.list_images(args["dataset"]))  # 5. Get a list of all the image paths in the dataset directory
data = []  # 6. Create an empty list to store the images
labels = []  # 7. Create an empty list to store the labels
# loop over the image paths 
for imagePath in imagePaths:  # 8. Loop over the image paths
    # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]  # 9. Extract the class label from the image path using the split method and the os.path.sep character as the separator
    # load the input image (224x224) and preprocess it
    image = load_img(imagePath, target_size=(224, 224))  # 10. Load the image using the load_img function from the keras.preprocessing.image module and specify the target size as (224, 224) pixels
    image = img_to_array(image)  # 11. Convert the image to a numpy array using the img_to_array function from the keras.preprocessing.image module
    image = preprocess_input(image)  # 12. Preprocess the image using the preprocess_input function from the mobilenet_v2 module
    # update the data and labels lists, respectively
    data.append(image)  # 13. Append the image to the data list
    labels.append(label)  # 14. Append the label to the labels list
# convert the data and labels to NumPy arrays
data = np.array(data, dtype="float32")  # 15. Convert the data list to a NumPy array using the np.array function and specify the data type as float32
labels = np.array(labels)  # 16. Convert the labels list to a NumPy array using the np.array function

[INFO] loading images...


In [5]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()  # 17. Create a LabelBinarizer object using the LabelBinarizer class from the sklearn.preprocessing module
labels = lb.fit_transform(labels)  # 18. Fit the LabelBinarizer object to the labels array and transform the labels array using the fit_transform method
labels = to_categorical(labels)  # 19. Convert the labels array to a one-hot encoded NumPy array using the to_categorical function from the keras.utils module
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, 
    test_size=0.20, stratify=labels, random_state=42)  # 20. Split the data into training and testing sets using the train_test_split function from the sklearn.model_selection module and specify the test size as 20% and the random state as 42
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")  # 21. Create an ImageDataGenerator object using the ImageDataGenerator class from the keras.preprocessing module and specify the rotation range, zoom range, width shift range, height shift range, shear range, horizontal flip, and fill mode as nearest

In [6]:
# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))  # 22. Load the MobileNetV2 network using the MobileNetV2 class from the keras.applications module and specify the weights as imagenet and include the top FC layers as false and the input tensor as the shape (224, 224, 3) pixels
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output  # 23. Get the output of the base model
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)  # 24. Average pool the output of the base model using the AveragePooling2D class from the keras.layers module and specify the pool size as (7, 7)
headModel = Flatten(name="flatten")(headModel)  # 25. Flatten the output of the AveragePooling2D layer using the Flatten class from the keras.layers module
headModel = Dense(128, activation="relu")(headModel)  # 26. Add a dense layer with 128 neurons and the ReLU activation function
headModel = Dropout(0.5)(headModel)  # 27. Add a dropout layer with a rate of 0.5
headModel = Dense(2, activation="softmax")(headModel)  # 28. Add a dense layer with 2 neurons and the softmax activation function
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)  # 29. Create a Model object using the Model class from the keras.models module and specify the inputs as the base model input and the outputs as the head model output
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
    layer.trainable = False  # 30. Loop over all layers in the base model and freeze them so they will *not* be updated during the first training process

In [ ]:
# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)  # 31. Create an Adam optimizer object using the Adam class from the keras.optimizers module and specify the learning rate as INIT_LR and the decay as INIT_LR / EPOCHS
model.compile(loss="binary_crossentropy", optimizer=opt,
    metrics=["accuracy"])  # 32. Compile the model using the binary_crossentropy loss function, the Adam optimizer, and the accuracy metric
# train the head of the network
print("[INFO] training head...")
H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)  # 33. Train the head of the network using the fit method from the keras.models module and specify the training data, validation data, batch size, and number of epochs

In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  0


In [2]:
tf.test.gpu_device_name()

''